In [1]:
import pandas as pd
%matplotlib inline
from pathlib import Path
from zipfile import ZipFile
import xlrd
import openpyxl

In [2]:
file_list = [i for i in Path('./Задание/Задание/').iterdir()]
file_list




[WindowsPath('Задание/Задание/json.json'),
 WindowsPath('Задание/Задание/txt.txt'),
 WindowsPath('Задание/Задание/Задание 1.xlsx'),
 WindowsPath('Задание/Задание/Задание 2 и 3.txt'),
 WindowsPath('Задание/Задание/Задание 4 (не обязательное).txt')]

In [3]:
dfs = pd.read_excel(
    file_list[2],
    engine='openpyxl',
    sheet_name=[1, 2]  # Replace 0 and 1 with the actual sheet indices you want to read
)

df_doctors = dfs[1]  # Assign the first DataFrame to df_doctors
df_report = dfs[2]   

In [4]:
(df_doctors.merge(
        df_doctors['ФИО']
        .str.upper()
        .str.extract(
        r'(?P<surname>[А-я]+)\s+(?P<name>[А-я]+)\s+(?P<patronymic>[А-я]+)'),
    left_index=True,right_index=True)
    
)

,ФИО,ДР,Дата Осмотра,Состояние,surname,name,patronymic
0,ФОМИЧЕВ ГОРДЕЙ НАУМОВИЧ,2000-02-13 00:00:00,2020-10-10,Болен\n,ФОМИЧЕВ,ГОРДЕЙ,НАУМОВИЧ
1,ХАРИТОНОВА ВЕРОНИКА ТАРАСОВНА,1977-11-07 00:00:00,2020-10-10,здоров,ХАРИТОНОВА,ВЕРОНИКА,ТАРАСОВНА
2,Сорокина Амелия Тимофеевна,12.09.1978,2020-10-10,здоров,СОРОКИНА,АМЕЛИЯ,ТИМОФЕЕВНА
3,Тарасов Гавриил Тихонович,1942-04-11 00:00:00,2020-10-20,Болен\n,ТАРАСОВ,ГАВРИИЛ,ТИХОНОВИЧ
4,Завьялова Елена Степановна,2002-09-11 00:00:00,2020-10-20,болен,ЗАВЬЯЛОВА,ЕЛЕНА,СТЕПАНОВНА
...,...,...,...,...,...,...,...
33472,Порывай Денис Геннадиевич,1975-01-09 00:00:00,2020-10-10,болен,ПОРЫВАЙ,ДЕНИС,ГЕННАДИЕВИЧ
33473,ЧЕРНЯЕВА ВЕРА ИОСИФОВНА,1970-02-16 00:00:00,2020-10-10,здоров,ЧЕРНЯЕВА,ВЕРА,ИОСИФОВНА
33474,Ильина Эмилия Платоновна\n,1931-01-08 00:00:00,2020-10-12,Болен,ИЛЬИНА,ЭМИЛИЯ,ПЛАТОНОВНА
33475,СКРЯБИНА МИЛАНА КОНСТАНТИНОВНА,02.10.1945,2020-10-10,здоров,СКРЯБИНА,МИЛАНА,КОНСТАНТИНОВНА


In [6]:
df_doctors.dtypes

ФИО                     object
ДР                      object
Дата Осмотра    datetime64[ns]
Состояние               object
dtype: object

In [7]:
df_doctors['ДР']
pd.to_datetime(df_doctors['ДР'])

0       2000-02-13
1       1977-11-07
2       1978-12-09
3       1942-04-11
4       2002-09-11
           ...    
33472   1975-01-09
33473   1970-02-16
33474   1931-01-08
33475   1945-02-10
33476   1937-02-03
Name: ДР, Length: 33477, dtype: datetime64[ns]

In [8]:
df_doctors_clear = (df_doctors.merge(
        df_doctors['ФИО']
        .str.upper()
        .str.extract(
        r'(?P<Фамилия>[А-я]+)\s+(?P<Имя>[А-я]+)\s+(?P<Отчество>[А-я]+)'),
    left_index=True,right_index=True)
    .assign(status=
        df_doctors['Состояние']
        .fillna('здоров')
        .str.extract(r'([А-я]+)',expand=False)
        .str.lower())
    .assign(birthday=
    pd.to_datetime(df_doctors['ДР']))
).drop(columns=['ФИО','Состояние','ДР']).rename(columns={'birthday':'ДР'})
df_doctors_clear

,Дата Осмотра,Фамилия,Имя,Отчество,status,ДР
0,2020-10-10,ФОМИЧЕВ,ГОРДЕЙ,НАУМОВИЧ,болен,2000-02-13
1,2020-10-10,ХАРИТОНОВА,ВЕРОНИКА,ТАРАСОВНА,здоров,1977-11-07
2,2020-10-10,СОРОКИНА,АМЕЛИЯ,ТИМОФЕЕВНА,здоров,1978-12-09
3,2020-10-20,ТАРАСОВ,ГАВРИИЛ,ТИХОНОВИЧ,болен,1942-04-11
4,2020-10-20,ЗАВЬЯЛОВА,ЕЛЕНА,СТЕПАНОВНА,болен,2002-09-11
...,...,...,...,...,...,...
33472,2020-10-10,ПОРЫВАЙ,ДЕНИС,ГЕННАДИЕВИЧ,болен,1975-01-09
33473,2020-10-10,ЧЕРНЯЕВА,ВЕРА,ИОСИФОВНА,здоров,1970-02-16
33474,2020-10-12,ИЛЬИНА,ЭМИЛИЯ,ПЛАТОНОВНА,болен,1931-01-08
33475,2020-10-10,СКРЯБИНА,МИЛАНА,КОНСТАНТИНОВНА,здоров,1945-02-10


In [9]:
merged_df = df_report.merge(df_doctors_clear)
_dates = list(df_report.columns[4:7])
_dates

[datetime.datetime(2020, 10, 10, 0, 0),
 datetime.datetime(2020, 10, 12, 0, 0),
 datetime.datetime(2020, 10, 20, 0, 0)]

In [13]:
# formatted_date = given_datetime.strftime('%d %b')

# Define a mapping of month abbreviations to their full names
month_mapping = {
    'Jan': 'янв',
    'Feb': 'фев',
    'Mar': 'мар',
    'Apr': 'апр',
    'May': 'май',
    'Jun': 'июн',
    'Jul': 'июл',
    'Aug': 'авг',
    'Sep': 'сен',
    'Oct': 'окт',
    'Nov': 'ноя',
    'Dec': 'дек'
}

# Replace the abbreviated month with the translated month name
def convert_data_rus(date):
    day = date.strftime('%d')
    month =  date.strftime('%b')
    return  (f'{day} {month_mapping[month]}')


'10 окт'

In [14]:
df_doctors_clear

,Дата Осмотра,Фамилия,Имя,Отчество,status,ДР
0,2020-10-10,ФОМИЧЕВ,ГОРДЕЙ,НАУМОВИЧ,болен,2000-02-13
1,2020-10-10,ХАРИТОНОВА,ВЕРОНИКА,ТАРАСОВНА,здоров,1977-11-07
2,2020-10-10,СОРОКИНА,АМЕЛИЯ,ТИМОФЕЕВНА,здоров,1978-12-09
3,2020-10-20,ТАРАСОВ,ГАВРИИЛ,ТИХОНОВИЧ,болен,1942-04-11
4,2020-10-20,ЗАВЬЯЛОВА,ЕЛЕНА,СТЕПАНОВНА,болен,2002-09-11
...,...,...,...,...,...,...
33472,2020-10-10,ПОРЫВАЙ,ДЕНИС,ГЕННАДИЕВИЧ,болен,1975-01-09
33473,2020-10-10,ЧЕРНЯЕВА,ВЕРА,ИОСИФОВНА,здоров,1970-02-16
33474,2020-10-12,ИЛЬИНА,ЭМИЛИЯ,ПЛАТОНОВНА,болен,1931-01-08
33475,2020-10-10,СКРЯБИНА,МИЛАНА,КОНСТАНТИНОВНА,здоров,1945-02-10


In [99]:


def fill_dates(dates:list=_dates):
    for date in dates:
        merged_df[date] =  merged_df[merged_df['Дата Осмотра'] == date].status
        merged_df[date].fillna('здоров',inplace=True)
    return merged_df

df_result = fill_dates().drop(columns=['Дата Осмотра','status'])
df_result[['Фамилия','Имя','Отчество','ДР']].value_counts()

Фамилия    Имя       Отчество    ДР        
ФЕДОСЕЕВ   БОГДАН    БОРИСОВИЧ   1971-07-19    3
ДАДАЕВА    ЯНА       РОМАНОВНА   1959-08-31    3
СОФРОНОВА  МАРИАННА  ЮРЬЕВНА     1941-02-23    3
МАРТЫНОВ   ИГОРЬ     ДМИТРИЕВИЧ  1965-01-07    3
РУСАКОВА   ЮЛИАНА    ТИМОФЕЕВНА  1994-02-22    3
                                              ..
КУЛЕШОВА   ЕВДОКИЯ   ПЕТРОВНА    2004-01-29    1
КУЛЕШОВ    МАРК      ЛЕОНИДОВИЧ  1984-02-08    1
КУЛАКОВА   ЛИЛИЯ     ТАРАСОВНА   2000-09-14    1
           ЗОЯ       ФИЛИППОВНА  1933-01-15    1
ЛАРИН      ИГНАТ     МИХАЙЛОВИЧ  1974-07-12    1
Name: count, Length: 8659, dtype: int64

In [105]:
import numpy as np
df_result[df_result['Фамилия'] == 'КУЛАКОВА']
df_result["ПОЛ"] = df_result['Отчество'].str[-2:].apply(lambda x: 'муж' if x=='ИЧ' else 'жен')
df_result = df_result.rename(
    columns=(
        {_dates[0]:convert_data_rus(_dates[0]),
        _dates[1]:convert_data_rus(_dates[1]),
        _dates[2]:convert_data_rus(_dates[2]),}))

df_piv = df_result[['10 окт','12 окт','20 окт','ПОЛ']]

df_piv.groupby('ПОЛ').apply(lambda group: group.drop(columns='ПОЛ').apply(pd.value_counts)).unstack()

grouped = df_piv.groupby('ПОЛ').apply(lambda group: group.drop(columns='ПОЛ').apply(lambda col: (col == 'болен').sum())).unstack()

df_res = grouped.reset_index()


df_res.columns = ['Дата', 'ПОЛ', 'Болен']


df_pivoted = df_res.pivot(index='Дата', columns='ПОЛ', values='Болен')
df_pivoted['Всего'] = df_pivoted['жен'] + df_pivoted['муж']
# Reset the index of the pivoted DataFrame
df_pivoted = df_pivoted

df_pivoted.T


Дата,10 окт,12 окт,20 окт
ПОЛ,,,
жен,2149,493,983
муж,2160,522,908
Всего,4309,1015,1891
